# Reachability Analysis for Split Computing Neural Networks

## 1. Import Packages

In [154]:
begin
	import Pkg
	Pkg.activate("..")
	Pkg.instantiate()
	
    push!(LOAD_PATH, "$(@__DIR__)/../src")
    push!(LOAD_PATH, "$(@__DIR__)/../Experiments")
    push!(LOAD_PATH, "$(@__DIR__)/../Benchmarks")
    Pkg.add(["Plots", "Distributions", "QuadGK", "ControlSystemsBase", "LinearAlgebra", "ReachabilityAnalysis"])

    
    using Plots
    using Distributions
    using Experiment
    using Models
    using QuadGK
    using ControlSystemsBase
    using LinearAlgebra
    using ReachabilityAnalysis
end

  Activating project at `~/COMP790_CPSEncryption`


   Resolving package versions...


  No Changes to `~/COMP790_CPSEncryption/Project.toml`
  No Changes to `~/COMP790_CPSEncryption/Manifest.toml`


## 2. Physical System and Parameters Setup

In [155]:
sys = benchmarks[:F1T]
const period = 0.02
const period_c = 0.02
const Dc =    0.005

0.005

In [156]:
sys_ideal = c2d(sys, period_c)
K_ideal = lqr(ControlSystemsBase.Discrete, sys_ideal.A, sys_ideal.B, I, I)

1×2 Matrix{Float64}:
 0.777044  1.05686

In [157]:
H = 1000
H_c= floor(Int, H * period / period_c)
x0 = 1.
u0 = 0.
n = 1
p = size(sys.A, 1)
q = size(sys.B, 2)
r = size(sys.C, 1)
x0_vec = fill(x0, p)
u_prev = fill(u0, q)
z0 = [x0_vec; u_prev]

3-element Vector{Float64}:
 1.0
 1.0
 0.0

In [158]:
sys_aug = let
    ϕ = ℯ^(period * sys.A)
    Γ₁ = matrix_integral(sys.A, sys.B, Dc, period)
    Γ₀ = matrix_integral(sys.A, sys.B, 0.0, period - Dc)
    ϕ_aug = [ϕ  Γ₁; zeros(q, p)  zeros(q, q)]
    Γ_aug = [Γ₀; I(q)]
    C_aug = [sys.C zeros(size(sys.C,1), q)]
    ss(ϕ_aug, Γ_aug, C_aug, sys.D, period)
end
K = lqr(ControlSystemsBase.Discrete, sys_aug.A, sys_aug.B, I, I)

1×3 Matrix{Float64}:
 0.55255  0.776143  0.221222

## 3. Calculate and Plot Reachable Trajectories and RMSE values

In [159]:
all_trajectories_z = []
all_trajectories_u = []
N = 50
M = 40
const xlim = 2
const ylim = 2
S = N - M + 1
for n in M:N
    z, u = evolve(sys_aug.A, sys_aug.B, K, H, z0, u0, n)
    push!(all_trajectories_z, z)
    push!(all_trajectories_u, u)
end
z_ideal, u_ideal = ideal_evolve(sys_ideal.A, sys_ideal.B, K_ideal, H_c, x0_vec, u0)

([[1.0, 1.0], [1.13, 1.0], [1.2104842811626617, 0.23821971019479748], [1.2109395643281746, -0.2312153538022929], [1.1630554280104048, -0.5054636664710889], [1.087888419525824, -0.6509518486763062], [0.9992374696670526, -0.7129089183817163], [0.9059705950330421, -0.7219660759876765], [0.8136258575246171, -0.6987221933727074], [0.725510451977095, -0.6568994304353266]  …  [3.3589918771826324e-60, -3.5914231101535814e-60], [2.9224458553242955e-60, -3.124669533820834e-60], [2.542634840923072e-60, -2.7185768415826075e-60], [2.2121853592248995e-60, -2.3652613383892774e-60], [1.924682217361726e-60, -2.0578639210441614e-60], [1.6745439627746157e-60, -1.7904169187575335e-60], [1.4569145274842585e-60, -1.557728239555656e-60], [1.2675689546410375e-60, -1.3552805734169746e-60], [1.1028313771736571e-60, -1.1791436953119557e-60], [9.595036562118792e-61, -1.0258981656384745e-60]], [[0.0], [-1.9349219361052146], [-1.1923650625526097], [-0.6965907141787419], [-0.36953998280125194], [-0.15737095705174173

In [160]:
traj_plot = plot(
    xlabel = "\$x_1\$",
    ylabel = "\$x_2\$",
    xguidefont = font(24),   # make x-axis label bigger
    yguidefont = font(24),   # make y-axis label bigger
    xtickfont = font(16),    # optional: make tick labels bigger
    ytickfont = font(16)
)
for trajectory in all_trajectories_z
    x_z = [point[1] for point in trajectory]
    y_z = [point[2] for point in trajectory]
    
    plot!(x_z, y_z, xlim=(0, xlim), ylim=(-ylim, ylim), label="", linecolor=:lightgray, linewidth=1)#, marker=:circle, markercolor=:yellow, markersize=2)
end
x_z_ideal = [point[1] for point in z_ideal]
y_z_ideal = [point[2] for point in z_ideal]
plot!(x_z_ideal, y_z_ideal, xlim=(0, xlim+0.2), ylim=(-ylim, ylim), label="", linecolor=:black, linewidth=2)
savefig(traj_plot, "bit_flipped.png")

"/Users/tingan/COMP790_CPSEncryption/src/bit_flipped.png"

In [161]:
rmse_list = compute_rmse_list(z_ideal, all_trajectories_z, period, period_c, H, S)
print(rmse_list)

[0.006220562322340152, 0.006193019180529001, 0.006278509432070791, 0.006191933677456921, 0.0064584618777238385, 0.006420520866029878, 0.008088227702539514, 0.013063099095675818, 0.01602282166400698, 0.03143556783259754, 0.12345716866120203]

In [162]:
plot_rmse_vs_bits(rmse_list, M, N, filename="rmse_bits_$(M)_to_$(N).png")

Plot saved to: rmse_bits_40_to_50.png


In [163]:
flip_bits_vec([5.4, 1.2], 61)

2-element Vector{Float64}:
 1.106144154220264e154
 2.685002663232074e-154